In [2]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_7d571d88e5bc4ad296acb578b62f3cf7 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='xMIIB1xwEzwb2z40TKGH8H-lJ9KD1pJiyc63TPB8gC0p',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu.cloud-object-storage.appdomain.cloud')

body = client_7d571d88e5bc4ad296acb578b62f3cf7.get_object(Bucket='detectionofphishingwebsites-donotdelete-pr-w1w9sasztldwrk',Key='dataset_website.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

dataset = pd.read_csv(body)
dataset.head()



,index,having_IPhaving_IP_Address,URLURL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,...,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,1,-1,1,1,1,-1,-1,-1,-1,-1,...,1,1,-1,-1,-1,-1,1,1,-1,-1
1,2,1,1,1,1,1,-1,0,1,-1,...,1,1,-1,-1,0,-1,1,1,1,-1
2,3,1,0,1,1,1,-1,-1,-1,-1,...,1,1,1,-1,1,-1,1,0,-1,-1
3,4,1,0,1,1,1,-1,-1,-1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
4,5,1,0,-1,1,1,-1,1,1,-1,...,-1,1,-1,-1,0,-1,1,1,1,1


In [3]:
#finding null values
dataset.isnull().any()

index                          False
having_IPhaving_IP_Address     False
URLURL_Length                  False
Shortining_Service             False
having_At_Symbol               False
double_slash_redirecting       False
Prefix_Suffix                  False
having_Sub_Domain              False
SSLfinal_State                 False
Domain_registeration_length    False
Favicon                        False
port                           False
HTTPS_token                    False
Request_URL                    False
URL_of_Anchor                  False
Links_in_tags                  False
SFH                            False
Submitting_to_email            False
Abnormal_URL                   False
Redirect                       False
on_mouseover                   False
RightClick                     False
popUpWidnow                    False
Iframe                         False
age_of_domain                  False
DNSRecord                      False
web_traffic                    False
P

In [4]:
#splitting the data into independent and dependent
x=dataset.iloc[:,1:31].values
y=dataset.iloc[:,-1].values
print(x,y)

[[-1  1  1 ...  1  1 -1]
 [ 1  1  1 ...  1  1  1]
 [ 1  0  1 ...  1  0 -1]
 ...
 [ 1 -1  1 ...  1  0  1]
 [-1 -1  1 ...  1  1  1]
 [-1 -1  1 ... -1  1 -1]] [-1 -1 -1 ... -1 -1 -1]


In [5]:
#splitting the data into train and test
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [6]:
#model building using logistic regression 
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(x_train,y_train)

LogisticRegression()

In [7]:
y_pred1=lr.predict(x_test)
from sklearn.metrics import accuracy_score
log_reg=accuracy_score(y_test,y_pred1)
log_reg

0.9167797376752601

In [8]:
import ibm_watson_machine_learning

In [9]:
from ibm_watson_machine_learning import APIClient
import json
import numpy as np

In [11]:
wml_credentials={
    "apikey":"Vhsx7vmc7em10KcOYiAVGpWhgtyR0-l5vzCXo8pT8EhI",
    "url":"https://eu-gb.ml.cloud.ibm.com"
}

In [12]:
wml_Client= APIClient(wml_credentials)
wml_Client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  ------------------------------  ------------------------
ID                                    NAME                            CREATED
e62d8aac-fb8d-4de6-9878-15c8397dda05  Detection of Phishing Websites  2022-03-06T15:55:29.901Z
------------------------------------  ------------------------------  ------------------------


In [13]:
SPACE_ID = "e62d8aac-fb8d-4de6-9878-15c8397dda05"

In [16]:
wml_Client.set.default_space(SPACE_ID)

'SUCCESS'

In [17]:
MODEL_NAME="Detection of Phishing Websites"
DEPLOYMENT_NAME="Detection of Phishing Websites deployment"
BEST_MODEL = lr

In [18]:
software_spec_uid = wml_Client.software_specifications.get_id_by_name("default_py3.8")

model_props = {
    wml_Client.repository.ModelMetaNames.NAME:MODEL_NAME,
    wml_Client.repository.ModelMetaNames.TYPE:'scikit-learn_0.23',
    wml_Client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
}

In [19]:
model_details = wml_Client.repository.store_model(
    model = BEST_MODEL,
    meta_props = model_props,
    training_data = x_train,
    training_target = y_train,
)

In [20]:
software_spec_uid

'ab9e1b80-f2ce-592c-a7d2-4f2344f77194'

In [21]:
model_uid  = wml_Client.repository.get_model_id(model_details)

In [23]:
model_uid

'1f9d41b1-078b-4455-af4e-09003bd09404'

In [25]:
deployment_props= {
    wml_Client.deployments.ConfigurationMetaNames.NAME:DEPLOYMENT_NAME,
    wml_Client.deployments.ConfigurationMetaNames.ONLINE:{}
}

In [26]:
deployment= wml_Client.deployments.create(artifact_uid=model_uid,meta_props=deployment_props)



#######################################################################################

Synchronous deployment creation for uid: '1f9d41b1-078b-4455-af4e-09003bd09404' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='8dd91582-83ea-48ee-b477-a85f1fbdf010'
------------------------------------------------------------------------------------------------




In [27]:
deployment_uid  = wml_Client.deployments.get_uid(deployment)
deployment_uid

'8dd91582-83ea-48ee-b477-a85f1fbdf010'

In [32]:
payload = {"input_data":[{"fields":x_test.columns.to_numpy().tolist(),"values":x_test.to_numpy().tolist()}]}

AttributeError: 'numpy.ndarray' object has no attribute 'columns'